# FASE 1

## Descarga de las imágenes que formarán el dataset de pruebas de la APP. Se descargarán las imagenes que contienen una sola persona y un número mínimo de XX keypoints para esa persona

## Creación de un fichero en formato JSON con la información de los keypoints de la imagenes descargadas para la posterior comprobación de la precisión de las estimación realizadas por los modelos


In [11]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt

import requests
from pathlib import Path


print('\n')
print('--------------------------------------------------')
print('Iniciado procesado del dataset')
print('--------------------------------------------------')

# Cargamos la información de los keypoints de las personas
personKeypointsVal2017File = './annotations_train-val/person_keypoints_val2017.json'
personKeypointsVal2017COCO = COCO(personKeypointsVal2017File)

# Obtenemos el identificador de la categoria de personas
personCategoryId = personKeypointsVal2017COCO.getCatIds(catNms=['person'])

# Obtenemos los identificadores de las imágenes en la categoria de personas
personCategoryImagesIds = personKeypointsVal2017COCO.getImgIds(catIds=personCategoryId)

# Obtenemos las anotaciones (contienen los keypoints) de las imágenes de personas
personCategoryAnnotationIds = personKeypointsVal2017COCO.getAnnIds(imgIds=personCategoryImagesIds, catIds=personCategoryId, iscrowd=None)  # iscrowd == 0
personCategoryAnnotations = personKeypointsVal2017COCO.loadAnns(personCategoryAnnotationIds)

# Array para almacenar los identificadores de las imágenes temporalmente
imageIdsTemp = []
# Array para almacenar los identificadores de las imágenes únicas
singlePersonImageIds = []
# Array para almacenar los elementos con anotaciones duplicadas
multiPersonImageIds = []
# Filtramos la lista para quedarnos con las imágenes que solo tienen una anotación (tienen sólo una persona en la imagen)
for annotationTemp in personCategoryAnnotations:
    # Metemos el id de cada imagen con alguna anotación en imageIdsTemp
    if annotationTemp['image_id'] not in imageIdsTemp:
        imageIdsTemp.append(annotationTemp['image_id'])
    else:
        # Metemos el id de cada imagen con mas de una anotación en multiPersonImageIds (sólo una vez)
        if annotationTemp['image_id'] not in multiPersonImageIds:
            multiPersonImageIds.append(annotationTemp['image_id'])

# Nos quedamos con los ids de las imágenes únicas que no tienen ningun id duplicado (anotación duplicada)
for idImageTemp in imageIdsTemp:
    if idImageTemp not in multiPersonImageIds:
        singlePersonImageIds.append(idImageTemp)


# Cargamos las anotaciones de esas imágenes con una anotación unica (tienen sólo una persona en la imagen)
singlePersonAnnotationIds = personKeypointsVal2017COCO.getAnnIds(imgIds=singlePersonImageIds, catIds=personCategoryId, iscrowd=None)
singlePersonAnnotations = personKeypointsVal2017COCO.loadAnns(singlePersonAnnotationIds)

# Array para almacenar los identificadores de las imagenes con una sola persona y al menos un keypoint de esa persona
singlePersonAnnotationFilteredIds = []
# Array para almacenar las lineas del fichero formato json que contendra la información de validación de las imánes del dataset
jsonFileLines = []

# Filtramos las imágenes que tienen al menos 1 keypoint en su anotacion
# - crear una lista de identificadores de las imágenes a descargar
# - crear una lista con las lineas con la información de keypoints de cada imagen para el fichero de validación
for annotationTemp in singlePersonAnnotations:
#    if annotationTemp['num_keypoints'] > 10 and annotationTemp['num_keypoints'] <= 17:          # 614 imágenes con 10-17 puntos
    if annotationTemp['num_keypoints'] > 1:                                                      # 887 imágenes con algún punto

        # Identificador de la imagen
        imageIdTemp = annotationTemp['image_id']

        # Añadimos el identificador de la imagen que vamos a descargar al array de identificadores filtrados
        singlePersonAnnotationFilteredIds.append(imageIdTemp)                                               

        # Añadimos la inea de información para el fichero json de validación al array de lineas
        # - añadimos el identificador de la imagen con el formato json correspondiente
        jsonFileLineTemp = '{"image_id":' + str(imageIdTemp) + ',"category_id":1,"keypoints":['
        # - añadimos los 17 keypoints de la imagen separados por comas
        for indexTemp in range(17):
            index_x = (indexTemp * 3)
            index_Y = (indexTemp * 3) + 1
            index_2 = (indexTemp * 3) + 2
            jsonFileLineTemp = jsonFileLineTemp + str(annotationTemp['keypoints'][index_x]) + ',' + str(annotationTemp['keypoints'][index_Y]) + ',' + str(annotationTemp['keypoints'][index_2])
            if indexTemp < 16:
                jsonFileLineTemp = jsonFileLineTemp + ','
        # Añadimos la finalización de la linea con la información de la imagen
        jsonFileLineTemp = jsonFileLineTemp + '], "score":0}'
        # Añadimos la linea con la información de la imagen para posteriormente generar el fichero en disco
        jsonFileLines.append(jsonFileLineTemp)
    
    
# Impresión de datos para seguimiento
print('\n')
print('Id de la categoría "person": ', personCategoryId)
print('Número de imágenes pertenecientes a la categoría "person": ', len(personCategoryImagesIds))
print('Anotaciones totales para las imágenes de la categoría "person" (por numero total de ids): ', len(personCategoryAnnotationIds))
print('Anotaciones totales para las imágenes de la categoría "person" (cargadas del fichero json): ', len(personCategoryAnnotations))
print('Número de imágenes con una sola persona: ', len(singlePersonImageIds))
print('Número de imágenes con una sola persona filtradas (tienen al menos un keypoint): ', len(singlePersonAnnotationFilteredIds))
print('Número de líneas de información de imágenes para el fichero de validación: ', len(jsonFileLines))


# Creamos la carpeta para el dataset
Path('./Dataset').mkdir(parents=True, exist_ok=True)
# Creamos la carpeta para las imágenes del dataset
Path('./Dataset/Images').mkdir(parents=True, exist_ok=True)

########################################################################################################################
# DESCOMENTAR PARA EJECUTAR LA DESCARGA DE LAS IMAGENES Y LA CREACION DEL FICHERO CSV CON LA INFORMACION DE KEYPOINTS
########################################################################################################################

print('\n')
print('Descargando imágenes del dataset...')
# Descargamos en local las imágenes filtradas (aquellas que tienen una sola persona con al menos un keypoint en sus anotaciones)
imagesTemp = personKeypointsVal2017COCO.loadImgs(singlePersonAnnotationFilteredIds)
for imageTemp in imagesTemp:
    img_data = requests.get(imageTemp['coco_url']).content
    with open('./Dataset/Images/' + imageTemp['file_name'], 'wb') as handler:
        handler.write(img_data)
print('Descargando imagenes del dataset...Finalizado')

########################################################################################################################

print('\n')
print('Creando fichero de validación...')

# Creamos un fichero json con los datos en formato adaptado para la comprobación de la validación de resultados
# Variable índice
indexTemp = 0
# Variable para el control de separadores (comas a añadir entre cada linea de información de cada imagen)
jsonLinesLength = len(jsonFileLines) - 1

# Recorremos la estructura con las lineas que hemos generado anteriormente para cada imagen filtrada
with open("./Dataset/DatasetValidationInfo.json", "wt") as f:
    # Escribimos el caracter de inicio de la estructura json
    f.write('[')
    # Escribimos cada linea generada de cada imagen anteriormente
    for jsonFileLineTemp in jsonFileLines:
        f.write(jsonFileLineTemp)
        # Escribimos los separadores cuando corresponde
        if indexTemp < jsonLinesLength:
            f.write(',')
        # Escribimos un retorno se linea
        f.write('\n')
        
        indexTemp = indexTemp + 1
    # Escribimos el caracter de finalización de la estructura json
    f.write(']')

print('Creando fichero de validación...Finalizado')

print('\n')
print('--------------------------------------------------')
print('Finalizado procesado del dataset')
print('--------------------------------------------------')





--------------------------------------------------
Iniciado procesado del dataset
--------------------------------------------------
loading annotations into memory...
Done (t=0.26s)
creating index...
index created!


Id de la categoría "person":  [1]
Número de imágenes pertenecientes a la categoría "person":  2693
Anotaciones totales para las imágenes de la categoría "person" (por numero total de ids):  11004
Anotaciones totales para las imágenes de la categoría "person" (cargadas del fichero json):  11004
Número de imágenes con una sola persona:  1045
Número de imágenes con una sola persona filtradas (tienen al menos un keypoint):  887
Número de líneas de información de imágenes para el fichero de validación:  887


Descargando imágenes del dataset...
Descargando imagenes...Finalizado


Creando fichero de validación...
Creando fichero de validación...Finalizado


--------------------------------------------------
Finalizado procesado del dataset
------------------------------------